In [6]:
import torch
torch.cuda.empty_cache()

In [1]:
import pandas as pd
import numpy as np
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from PIL import Image

In [2]:
from torch.nn.modules import batchnorm
bs = 1
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout2d(0.1)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout2d(0.2)
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128)
        )
        self.conv6 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout2d(0.3)
        )
        self.conv7 = nn.Sequential(
            nn.Conv2d(128,256,kernel_size=4,padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Dropout2d(0.3)
        )
        self.conv8 = nn.Sequential(
            nn.Conv2d(256,256,kernel_size=3,padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(256)
        )

        self.fc = nn.Linear(256, 10)  # number of classes =10

    def forward(self, x):
        x = self.conv8(self.conv7(self.conv6(self.conv5(self.conv4(self.conv3(self.conv2(self.conv1(x))))))))        
        x = F.avg_pool2d(x, kernel_size=x.shape[2:])
        x = x.view(x.shape[0], -1)

        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [3]:
def train(model, dataloader, epoch, optimizer, criterion):
    
    model.train()
    train_loss = []
    train_acc = []
    for batch_idx, (data, target) in enumerate(dataloader):
        data, target = data.cuda(), target.cuda()
        output = model(data)
        
        optimizer.zero_grad()
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        pred_cls = output.max(1)[1]
        correct = pred_cls.eq(target.long().data).cpu().sum()
        
        train_acc.append(correct.item()/data.shape[0])
        train_loss.append(loss.item())
        
        if batch_idx % 50 == 0:
            print('\rTrain Epoch: {} [({:.0f}%)]\tLoss: {:.3f}\tAcc: {:.2f}%'.format(
            epoch+1, 100. * batch_idx / len(dataloader), np.mean(train_loss), 100*np.mean(train_acc)), end="")
    print()

In [4]:
def evaluate_model(model,testdataloder):
  
  val_acc=[]
  model.eval()

  with torch.no_grad():
    for data, target in testdataloder:
      data, target = data.cuda(), target.cuda()
      output=model(data)
      pred_cls=output.max(1)[1]
      correct=pred_cls.eq(target.long().data).cpu().sum()
      val_acc.append(correct.item()/data.shape[0])
  print("val_acc :{:2f}%".format(100*np.mean(val_acc)))
  return np.mean(val_acc)

In [5]:
classes = {0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}


In [6]:
def main():
    
    #Loading the dataset 
    
    cifar_train_dataset=torchvision.datasets.CIFAR10(
        root = './data',
        transform = transforms.Compose([
                transforms.Resize([32, 32]),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(), 
                transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5, 0.5, 0.5])
        ]),
        download=True,
        train=True
    )
    
    # Defining the dataloader
    
    dataloader = torch.utils.data.DataLoader(cifar_train_dataset, batch_size=64, shuffle = True, num_workers=4)
    
    cifar_test_dataset=torchvision.datasets.CIFAR10(
        root = './data',
        transform = transforms.Compose([
                transforms.Resize([32, 32]),
                transforms.ToTensor(), 
                transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5, 0.5, 0.5])
        ]),
        download=True,
        train=False
    )
    
    testdataloader = torch.utils.data.DataLoader(cifar_test_dataset, batch_size=64, shuffle = False, num_workers=6)
    
    # defining the model
    model = NeuralNet()
    # defining the optimizer
    optimizer = Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    criterion=nn.NLLLoss()
    if torch.cuda.is_available():
        model = model.cuda()
        criterion = criterion.cuda()
    
    print("st of model",model)

    n_epoch=100
    best=0
    for epoch in range (n_epoch):
      train(model,dataloader,epoch,optimizer,criterion)
      val_acc=evaluate_model(model,testdataloader)
      if val_acc>best:
        best=val_acc
        torch.save(model.state_dict(),"best.ckpt")

    print("Done")




In [ ]:
main()